코드 참고: \
https://github.com/kairess/mental-health-chatbot

* Requirements
> Main CSV data file \
> Edited CSV data file (w/ answer encoded column) \
> ipynb file for python \
> py file for streamlit (localhost)

In [2]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 87.5 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=64389a2af4838371596f0792030721a7cd7c651f5f0606ac1e556df030fa8813
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np
import pandas as pd
import tensorflow as tf
import sentencepiece as spm
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

import re
import os
import random
import math

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

print(tf.__version__)

2.9.2


In [4]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]
embeddings=model.encode(sentences)

print(embeddings)

[[-0.37510476 -0.77338415  0.5927711  ...  0.57923514  0.3268347
  -0.65089625]
 [-0.09361722 -0.18191545 -0.19230829 ... -0.03165793  0.3041255
  -0.26793614]]


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
!wget https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv
df = pd.read_csv("ChatbotData.csv")
print(df.count())
df.sample(n=100)

--2023-01-27 01:55:57--  https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 889842 (869K) [text/plain]
Saving to: ‘ChatbotData.csv’

ChatbotData.csv     100%[===================>] 868.99K  --.-KB/s    in 0.005s  

2023-01-27 01:55:58 (186 MB/s) - ‘ChatbotData.csv’ saved [889842/889842]

Q        11823
A        11823
label    11823
dtype: int64


,Q,A,label
304,그 사람이 나 좋아해줬으면 좋겠다,저도 간절히 기도 할게요.,0
1493,막말 할거 같은데,입을 닫고 듣는 연습을 해보세요.,0
5785,나를 좀 놓아달라는 여친. 혹시 가능성은 있을까? 죽을 것 같네.,당신에게는 선택권이 없네요.,1
5151,혼자 영화나 봐야겠다.,편하고 좋죠.,0
5208,환승이 될까,환승은 30분 안에,0
...,...,...,...
9825,사귀고 나서 환장이 깨졌어,환장을 갖지 말고 현실을 바라보세요.,2
8167,죽어가는.,기운을 내요. 그런 말은 너무 슬퍼요.,1
1828,뭐부터 바꿔야 할까,당신 그대로의 모습이 보기 좋아요.,0
4431,참견 좀 안했으면,서툰 애정표현일 수도 있어요.,0


In [15]:
df.loc[0, 'Q']
model.encode(df.loc[0, 'Q'])

array([-1.90726951e-01,  3.21475983e-01,  5.98737538e-01,  4.46178555e-01,
       -2.59639084e-01, -5.52718043e-01, -5.07271647e-01,  3.00774664e-01,
        3.17947656e-01,  5.00524879e-01, -8.11978638e-01,  5.53099215e-02,
        4.61658955e-01,  5.14434755e-01,  1.05695464e-01,  1.03816950e+00,
        1.60535708e-01, -2.70635217e-01,  4.00858968e-02, -3.95542942e-02,
        4.88820881e-01, -3.36692959e-01,  4.04013067e-01,  6.36627376e-01,
        2.66322345e-01,  7.85852194e-01, -4.81335878e-01,  5.80019593e-01,
       -2.60483380e-02, -3.82106811e-01, -7.81929672e-01,  2.49330714e-01,
       -2.72424877e-01,  1.29058361e-01,  4.07537729e-01, -3.52677673e-01,
        4.06023502e-01, -2.16337800e-01,  6.45365894e-01,  4.36917067e-01,
        2.81540304e-01,  8.01312387e-01,  1.33056954e-01,  4.37608272e-01,
       -2.65099019e-01, -5.56235969e-01, -1.86636209e-01, -4.79364425e-01,
       -8.03265154e-01,  2.62957603e-01,  8.46057713e-01, -2.75467396e-01,
       -6.12055600e-01,  

In [16]:
#유저 대화내용 인코딩
#로딩시간: 2분 30초

df['embedding'] = pd.Series([[]] * len(df))
df['embedding'] = df['Q'].map(lambda x: list(model.encode(x)))
df.head

<bound method NDFrame.head of                              Q                         A  label  \
0                       12시 땡!                하루가 또 가네요.      0   
1                  1지망 학교 떨어졌어                 위로해 드립니다.      0   
2                 3박4일 놀러가고 싶다               여행은 언제나 좋죠.      0   
3              3박4일 정도 놀러가고 싶다               여행은 언제나 좋죠.      0   
4                      PPL 심하네                눈살이 찌푸려지죠.      0   
...                        ...                       ...    ...   
11818           훔쳐보는 것도 눈치 보임.        티가 나니까 눈치가 보이는 거죠!      2   
11819           훔쳐보는 것도 눈치 보임.             훔쳐보는 거 티나나봐요.      2   
11820              흑기사 해주는 짝남.                    설렜겠어요.      2   
11821  힘든 연애 좋은 연애라는게 무슨 차이일까?  잘 헤어질 수 있는 사이 여부인 거 같아요.      2   
11822               힘들어서 결혼할까봐        도피성 결혼은 하지 않길 바라요.      2   

                                               embedding  
0      [-0.19072695, 0.32147598, 0.59873754, 0.446178...  
1      [0.12320559, 0.64236844, 0.68536025, -0.

In [26]:
text = '너는 누구니?'
embedding = model.encode(text)

In [29]:
df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())
df.head()

,Q,A,label,embedding,distance,similarity
0,12시 땡!,하루가 또 가네요.,0,"[-0.19072695, 0.32147598, 0.59873754, 0.446178...",0.205038,0.205038
1,1지망 학교 떨어졌어,위로해 드립니다.,0,"[0.12320559, 0.64236844, 0.68536025, -0.429123...",0.132061,0.132061
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0,"[0.02435763, -0.2239355, 0.22917706, -0.450131...",0.212928,0.212928
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0,"[0.036285352, -0.21048272, 0.072693996, -0.540...",0.203822,0.203822
4,PPL 심하네,눈살이 찌푸려지죠.,0,"[0.022030873, 0.70552176, -0.1715724, 0.165971...",0.152738,0.152738


In [30]:
ans = df.loc[df['similarity'].idxmax()]

print('구분', ans['label'])
print('유사한 질문', ans['Q'])
print('챗봇 답변', ans['A'])
print('유사도', ans['similarity'])

구분 0
유사한 질문 너 누구니?
챗봇 답변 저는 위로해드리는 로봇이에요.
유사도 0.9762445688247681
